In [209]:
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Subset
from torch import nn, optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchsummary import summary
import seaborn as sns
from copy import deepcopy
from tqdm import tqdm

# Req for package
import sys
sys.path.append("../")
from SkinLearning.NN.Helpers import train, test as t, DEVICE, getParameterLoss, setSeed
from SkinLearning.NN.Models import MultiTemporal
from SkinLearning.Utils.Dataset import getDataset, getSplit
from SkinLearning.Utils.Plotting import plotParameterBars


torch.backends.cudnn.benchmark = True

In [12]:
setSeed()

In [132]:
class SignalAutoencoder(nn.Module):
    def __init__(self, encoding_dim=8):
        super(SignalAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, encoding_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU()
        )

    def forward(self, x):
        batch_size = x.shape[0]
        x1 = x[:, 0, :].reshape(batch_size, 1, -1)
        x2 = x[:, 1, :].reshape(batch_size, 1, -1)
        
        encoded1 = self.encoder(x1)
        encoded2 = self.encoder(x2)
        
        decoded1 = self.decoder(encoded1)
        decoded2 = self.decoder(encoded2)

        return torch.concat([encoded1, encoded2], dim=1), torch.concat([decoded1, decoded2], dim=1)

In [160]:
# In theory wont work
class SiameseRNN(nn.Module):
    def __init__(self, input_size=16, hidden_size=512):
        super(SiameseRNN, self).__init__()
        self.hidden_size = hidden_size
        # num_layers > 5 reduces performance
        self.ae1 = SignalAutoencoder()
        self.ae2 = SignalAutoencoder()
        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=2, batch_first=True)
                
        self.fc = nn.Sequential(
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 6)
        
        )
    
    def forward(self, x):
        batch_size = x.shape[0]
        x = encoder(x)[0]
        
        x1 = x[:, 0, :].reshape(batch_size, 1, -1)
        x2 = x[:, 1, :].reshape(batch_size, 1, -1)
        
        o1, h1 = self.rnn(x1)  # Add a batch dimension
        o2, h2 = self.rnn(x2)  # Add a batch dimension
        
        #out = torch.cat([h1[-1], h2[-1]], dim=1)
        out=h1[-1]
        out = out.reshape(batch_size, -1)
        out = self.fc(out)
        return out

In [4]:
dataset, scaler = getDataset()

100%|█████████████████████████████████████████████████████████████████████████████| 2241/2241 [00:09<00:00, 229.49it/s]


In [5]:
train_loader, test_loader = getSplit(dataset)

In [163]:
net = SiameseRNN()

In [164]:
train(train_loader, net, val_loader=test_loader, LR=0.001, epochs=1500, early_stopping=True)

Using: cuda


  4%|▉                         | 2/56 [00:00<00:00, 102.55batch/s, counter=25, epoch=51, lastLoss=0.186, valLoss=0.182]


KeyboardInterrupt: 

In [184]:
class CAE(nn.Module):
    def __init__(self, hidden_size=256, single_fc=True, out="f_hidden"):
        super(CAE, self).__init__()
        
        self.enc = nn.Sequential(
            nn.Conv1d(2, 128, kernel_size=5, padding=1, bias=False),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=5, stride=2),

            nn.Conv1d(128, 256, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),

            nn.Conv1d(256, 512, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
        )
        
        self.dec = nn.Sequential(
            
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.ReLU(),
            nn.ConvTranspose1d(512, 256, kernel_size=5, padding=1, bias=False),
            
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.ReLU(),
            nn.ConvTranspose1d(256, 128, kernel_size=3, padding=1, bias=False),
            
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.ReLU(),
            nn.ConvTranspose1d(128, 2, kernel_size=3, padding=1, bias=False),
            
        )
        
    def forward(self, x):
        batch_size = x.shape[0]
        
        enc1 = self.enc(x)
        dec1 = self.dec(enc1)
            
        return enc1, dec1

In [185]:
encoder = CAE()

In [186]:
train(train_loader, encoder, val_loader=test_loader, LR=0.0001, epochs=1500, early_stopping=True)

Using: cuda


100%|█████████████████████| 56/56 [00:00<00:00, 161.50batch/s, counter=0, epoch=67, lastLoss=0.000132, valLoss=7.04e-5]


100%|█████████████████████| 56/56 [00:00<00:00, 169.59batch/s, counter=3, epoch=135, lastLoss=2.15e-5, valLoss=2.41e-5]


100%|█████████████████████| 56/56 [00:00<00:00, 168.73batch/s, counter=5, epoch=203, lastLoss=3.15e-6, valLoss=3.76e-6]


100%|████████████████████| 56/56 [00:00<00:00, 163.51batch/s, counter=27, epoch=271, lastLoss=3.64e-6, valLoss=2.03e-6]


...

100%|█████████████████████| 56/56 [00:00<00:00, 174.74batch/s, counter=2, epoch=543, lastLoss=1.74e-7, valLoss=2.66e-7]


100%|████████████████████| 56/56 [00:00<00:00, 169.17batch/s, counter=45, epoch=611, lastLoss=9.51e-8, valLoss=1.72e-7]


100%|████████████████████| 56/56 [00:00<00:00, 167.20batch/s, counter=49, epoch=615, lastLoss=1.03e-7, valLoss=7.24e-8]

Early stopping after 616 epochs
Average train loss: 8.447184908910218e-05
Average validation loss: 3.1040995137877146e-05


([0.0844115265977702,
  0.00499564955576456,
  0.0031544263724104632,
  0.0023758517303836663,
  0.0018608077703642526,
  0.001541190001132366,
  0.001305987811065279,
  0.0012197478364604259,
  0.0010039584272557736,
  0.0009443823281409485,
  0.000949007245903236,
  0.0008195393280142785,
  0.0007020178943223852,
  0.0007280986527413395,
  0.0005692042785605216,
  0.0005910676747069894,
  0.0005443428022512567,
  0.0004654561110198431,
  0.0004995587847328611,
  0.0005306084739069254,
  0.00049250910160481,
  0.00040520072434446774,
  0.00042119490951465977,
  0.0003352799937731886,
  0.0003669122925202828,
  0.0003319588234132555,
  0.0003146752600774302,
  0.0003187810465793258,
  0.00027911947654502,
  0.00028258593530543815,
  0.00035180378550389184,
  0.0004397859177385856,
  0.00039490563566297557,
  0.00035268515057396144,
  0.0003010509229040638,
  0.0002664322504902624,
  0.0002647368933789299,
  0.00023573117713177844,
  0.00019811816933465058,
  0.00021477501858108944,
  0

In [182]:
def train(
    train_loader,
    net,
    LR=0.1,
    epochs=2000,
    val_loader=None,
    early_stopping=False,
    patience=50,
    optimizer=optim.Adam,
    plot=False,
    cluster=False
):
    net.to(DEVICE)
    optimizer = optimizer(net.parameters(), lr=LR)
    criterion = nn.MSELoss()
    val_losses = []        
    losses = []
    best_val_loss = 1e10
    counter = 0
    
    global loss
    loss = 0
    
    if plot:
        _, ax = plt.subplots(1, 1)

    print(f"Using: {DEVICE}")

    def processBatch(ittr):
        global loss
        loss = 0
        
        for _, data in enumerate(ittr):
            inp, out = data['input'].to(DEVICE), data['output'].to(DEVICE)

            optimizer.zero_grad()
            predicted = torch.Tensor(net(inp)[1])

            cost = criterion(inp, predicted)
            loss += cost.item()
            cost.backward()
            optimizer.step()
    
    for epoch in range(epochs):
        net.train()
        
        if plot or cluster:
            processBatch(train_loader)
        else:
            with tqdm(train_loader, unit="batch") as it:
                if epoch > 0:
                    it.set_postfix(lastLoss=losses[-1], valLoss=0 if len(val_losses) \
                         == 0 else val_losses[-1], counter=counter, epoch=epoch+1/epochs)
                processBatch(it)
        
        loss /= len(train_loader)
        losses.append(loss)

        if val_loader:
            val_loss = 0
            net.eval()
            for idx, data in enumerate(val_loader):
                inp, out = data['input'].to(DEVICE), data['output'].to(DEVICE)

                predicted = torch.Tensor(net(inp)[1])
                cost = criterion(inp, predicted)
                val_loss += cost.item()
            val_loss /= len(val_loader)  
            val_losses.append(val_loss)
            
            if plot:
                update_plot(losses, val_losses, ax, "Training", epoch)
            if cluster:
                print(f"Epoch {epoch+1}/{epochs}:")
                print(f"    Training loss: {losses[-1]}")
                print(f"    Validation loss: {val_losses[-1]}")
                print(f"    Stagnation counter: {counter}\n")

            if early_stopping:
                if val_losses[-1] < best_val_loss:
                    best_val_loss = val_loss
                    counter = 0
                else:
                    counter += 1
                if counter >= patience:
                    print(f"Early stopping after {epoch + 1} epochs")
                    break
         
    print(f"Average train loss: {np.sum(losses)/epochs}")
    print(f"Average validation loss: {np.sum(val_losses)/epochs}")
    
    return losses, val_losses


In [196]:
# Remove one FC LAyer
class MultiLSTM(nn.Module):
    def __init__(self, hidden_size=256, single_fc=True, out="f_hidden"):
        super(MultiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.out = out
        
        self.cnn = encoder
        
        self.lstm = nn.LSTM(15, hidden_size, batch_first=True, num_layers=1, bidirectional=True)
        
        input_tensor = torch.zeros(32, 512, 15)
        output, hidden = self.lstm(input_tensor)
        
        fc_in = hidden_size
        if out == 'output':
            fc_in = output.shape[1] * output.shape[2]
        elif out == 'f-output':
            fc_in = output.shape[2]
        elif fc_in == 'hidden' or out == 'cell':
            out = hidden_size * output.shape[2]
        elif out == 'f-hiden' or out == 'f-cell':
            fc_in = output.shape[2]
        elif out == 'h+o' or out == 'h+c' :
            fc_in = output.shape[1]
        
        if single_fc:
            self.fc = nn.Linear(fc_in*2, 6)
        else:
            self.fc = nn.Sequential(
                nn.Linear(fc_in*2, 128),
                nn.ReLU(),
                nn.Linear(128 , 64),
                nn.ReLU(),
                nn.Linear(64, 6),   
            )

    def forward(self, x):
        batch_size = x.shape[0]
        x = encoder(x)[0]
        
        #h0 = torch.zeros(1, batch_size, 256).to(x.device)
        #c0 = torch.zeros((1, batch_size, self.hidden_size)).to(x.device)
        o, (h, c) = self.lstm(x)
        
        if self.out == "f_hidden":
            x = h[-1].reshape(batch_size, -1)
        elif self.out == "hidden":
            x = h.reshape(batch_size, -1)
        elif self.out == "f_output":
            x = o[:, -1, :].reshape(batch_size, -1)
        elif self.out == "output":
            x = o.reshape(batch_size, -1)
        elif self.out == "f_cell":
            x == c[-1].reshape(batch_size, -1)
        elif self.out == "cell":
            x == c.reshape(batch_size, -1)
        elif self.out == "h+c":
            x = torch.concat([h[-1], c[-1]], dim=1).view(batch_size, -1)
        elif self.out == "h+o":
            x = torch.concat([h[-1], o[:, -1, :]], dim=1).view(o.size(0), -1)
            
        x = self.fc(x)
        return x

In [206]:
test = MultiLSTM(out="f_output", single_fc=False)

In [207]:
traint_loss, valt_loss = train(train_loader, test, val_loader=test_loader, LR=0.0001, epochs=1500, early_stopping=True)

Using: cuda


100%|█████████████████████████| 56/56 [00:02<00:00, 26.29batch/s, counter=5, epoch=67, lastLoss=0.0527, valLoss=0.0589]


100%|███████████████████████| 56/56 [00:02<00:00, 26.06batch/s, counter=14, epoch=135, lastLoss=0.0448, valLoss=0.0513]


100%|███████████████████████| 56/56 [00:02<00:00, 25.33batch/s, counter=49, epoch=170, lastLoss=0.0432, valLoss=0.0487]


Early stopping after 171 epochs
Average train loss: 0.00643280007087049
Average validation loss: 0.006802819130900834


In [210]:
t(test_loader, test, scaler)

(93.37990760803223,
 array([94.31275 , 88.29201 , 99.73189 , 99.81301 , 85.045235, 93.08455 ],
       dtype=float32),
 0.0470153254767259)